In [1]:
import math
import mip
import pandas as pd
import datetime
from pandas import ExcelFile

In [2]:
#acquisizione dei dati dal foglio elettronico
p_v = pd.read_excel('file di dati.xls',sheet_name='porzioni di viaggio')
t_p = pd.read_excel('file di dati.xls',sheet_name='tempi di percorrenza a vuoto')

In [3]:
#funzione per ricavare i tempi di percorrenza a vuoto tra due tratte
nomi = {t_p.iloc[0][i]:i for i in range(1,17)}

def t_trasf(i,j):
    a = nomi[p_v.iloc[j][2]]
    b = nomi[p_v.iloc[i][3]]
    #matrice riempita per metà...
    if(math.isnan(t_p.iloc[a][b])):
        return datetime.timedelta(minutes=t_p.iloc[b][a])
    return datetime.timedelta(minutes=t_p.iloc[a][b])

In [4]:
#inizializzazione delle variabili
k = 33 #numero autobus
n_t = 156 #numero tratte
dim = n_t * 2 + 2 #numero nodi del grafo
A = [[0]*dim for i in range(dim)]
c = [[0]*dim for i in range(dim)]
u = [[0]*dim for i in range(dim)]
b = [0 for i in range(dim)]
b[dim-2] = (-1)*k #nodo radice
b[dim-1] = k      #nodo pozzo

In [5]:
#matrice di compatibilità
for i in range(1, n_t+1):
    for j in range(i+1, n_t+1):
        t = t_trasf(i,j)
        if((p_v.iloc[i,5]+t)<=p_v.iloc[j,4]):
            A[i+n_t-1][j-1]=1
            u[i+n_t-1][j-1]=1 

In [6]:
#archi (i -> i') 
for i in range(n_t):
    for j in range(n_t):
        if(i==j):
            A[i][j+n_t]=1
            u[i][j+n_t]=1
            

#archi (r -> i)
for j in range(n_t):
    A[dim-2][j]=1
    u[dim-2][j]=1
    
#archi (i' -> p)
for i in range(n_t):
    A[i+n_t][dim-1]=1
    u[i+n_t][dim-1]=1
    
#arco (r -> p)
A[dim-2][dim-1]=1
u[dim-2][dim-1]=k     

In [7]:
#matrice dei costi
for i in range(n_t):
    for j in range(n_t):
        if(i==j):
            c[i][j+n_t]=(-1)*(p_v.iloc[i+1][1])

In [8]:
m = mip.Model()
x = [m.add_var(var_type=mip.INTEGER) for h in range(dim*dim)]

In [9]:
#Funzione obbiettivo
m.objective = mip.minimize(mip.xsum(c[i][j]*x[(i*dim)+j] for j in range(dim) for i in range(dim) if A[i][j]>0))

In [10]:
#Vincoli di bilancio
for i in range(dim):
    
    BS = []
    for k in range(dim):
        if(A[k][i]>0):
            BS.append(k)
    FS = []
    for k in range(dim):
        if(A[i][k]>0):
            FS.append(k)
            
    m.add_constr(  mip.xsum(x[(j*dim)+i] for j in BS)  -  mip.xsum(x[(i*dim)+j] for j in FS)  ==  b[i])

In [11]:
for i in range(dim):
    for j in range(dim): 
        if(A[i][j]>0):
            m.add_constr(x[(i*dim)+j]>= 0)

In [12]:
for i in range(dim):
    for j in range(dim): 
        if(A[i][j]>0):
            m.add_constr(x[(i*dim)+j]<= u[i][j])

In [13]:
#ottimizzazione
m.optimize()

#for i in range(dim):
    #print([int(x[(i*dim)+j].x) for j in range(dim)])

<OptimizationStatus.OPTIMAL: 0>

In [14]:
print('optimal solution cost: {} '.format(m.objective_value))

optimal solution cost: -5279.0 
